In [1]:
import argparse

import torch
from torchvision import utils
from model import Generator
from tqdm import tqdm
import easydict
import numpy as np
from matplotlib import pyplot as plt
import cv2


C:\Users\melon\anaconda3\lib\site-packages\torch\utils\cpp_extension.py:274: UserWarning: Error checking compiler version for cl: [WinError 2] 지정된 파일을 찾을 수 없습니다
  warnings.warn('Error checking compiler version for {}: {}'.format(compiler, error))


In [2]:
from easydict import EasyDict as edict
args = edict({'size':1024, #웬만하면 1024x1024로 고정
              'channel_multiplier':2,
              'truncation_mean': 4096,
              'truncation': 0.7, #0.7~0.8주면 artifact줄어듦(정상적인 이미지가 나올 것임)
              'sample': 1,
              'ckpt':'./stylegan2-ffhq-config-f.pt'})

In [3]:
device='cpu'
args.latent = 512
args.n_mlp = 8

g_ema = Generator(
    args.size, args.latent, args.n_mlp, channel_multiplier=args.channel_multiplier
).to(device)
checkpoint = torch.load(args.ckpt, map_location='cpu')

g_ema.load_state_dict(checkpoint["g_ema"])

if args.truncation < 1:
    with torch.no_grad():
        mean_latent = g_ema.mean_latent(args.truncation_mean)
else:
    mean_latent = None

In [14]:
# latent vector projecting
!python projector.py --ckpt stylegan2-ffhq-config-f.pt --w_plus --size 1024 --step 500 trump_face.jpg
#step : projecting 퀄리티 조정(커질수록 퀄리티 좋아짐, 시간은 많이 걸림)

Setting up Perceptual loss...
Loading model from: C:\Users\melon\pytorchtutorial_cnn\9_GAN2\stylegan2-pytorch\lpips\weights\v0.1\alex.pth
...[net-lin [alex]] initialized
...Done


C:\Users\melon\anaconda3\lib\site-packages\torch\utils\cpp_extension.py:274: UserWarning: Error checking compiler version for cl: [WinError 2] 지정된 파일을 찾을 수 없습니다
  warnings.warn('Error checking compiler version for {}: {}'.format(compiler, error))
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to C:\Users\melon/.cache\torch\hub\checkpoints\alexnet-owt-4df8aa71.pth

  0%|          | 0.00/233M [00:00<?, ?B/s]
  0%|          | 640k/233M [00:00<00:37, 6.44MB/s]
  1%|          | 1.32M/233M [00:00<00:36, 6.64MB/s]
  1%|1         | 2.47M/233M [00:00<00:31, 7.64MB/s]
  2%|1         | 3.66M/233M [00:00<00:28, 8.59MB/s]
  2%|2         | 4.75M/233M [00:00<00:26, 9.14MB/s]
  2%|2         | 5.59M/233M [00:00<00:26, 9.00MB/s]
  3%|2         | 6.42M/233M [00:00<00:27, 8.65MB/s]
  3%|3         | 7.23M/233M [00:00<00:32, 7.18MB/s]
  4%|3         | 8.23M/233M [00:00<00:29, 7.90MB/s]
  4%|3         | 9.03M/233M [00:01<00:30, 7.68MB/s]
  4%|4         | 9.88M/233M [00:01<00:29, 

In [15]:
trump_latent = torch.load('./trump_face.jpgproj.pt')

with torch.no_grad():
    g_ema.eval()
    for i in tqdm(range(1)):
        latent = trump_latent['trump_face.jpg']['latent']
        noise = trump_latent['trump_face.jpg']['noise']
        

        sample, _ = g_ema(
            [latent],
            input_is_latent=True,
            noise=noise
        )

        utils.save_image(
            sample,
            f"sample/trump/Trump.png",
            nrow=1,
            normalize=True,
            range=(-1, 1),
        )

 55%|#####5    | 129M/233M [00:12<00:10, 10.6MB/s]
 56%|#####5    | 130M/233M [00:12<00:09, 11.0MB/s]
 56%|#####6    | 131M/233M [00:12<00:09, 10.9MB/s]
 57%|#####6    | 132M/233M [00:13<00:09, 10.8MB/s]
 57%|#####7    | 133M/233M [00:13<00:09, 10.8MB/s]
 58%|#####7    | 134M/233M [00:13<00:09, 11.0MB/s]
 58%|#####8    | 135M/233M [00:13<00:09, 10.9MB/s]
 59%|#####8    | 137M/233M [00:13<00:09, 11.0MB/s]
 59%|#####9    | 138M/233M [00:13<00:09, 10.9MB/s]
 59%|#####9    | 139M/233M [00:13<00:09, 10.6MB/s]
 60%|#####9    | 140M/233M [00:13<00:09, 10.4MB/s]
 60%|######    | 141M/233M [00:13<00:09, 10.7MB/s]
 61%|######    | 142M/233M [00:14<00:08, 11.1MB/s]
 61%|######1   | 143M/233M [00:14<00:08, 11.0MB/s]
 62%|######1   | 144M/233M [00:14<00:08, 11.1MB/s]
 62%|######2   | 145M/233M [00:14<00:09, 10.2MB/s]
 63%|######2   | 146M/233M [00:14<00:08, 10.4MB/s]
 63%|######3   | 147M/233M [00:14<00:08, 10.5MB/s]
 64%|######3   | 148M/233M [00:14<00:08, 10.5MB/s]
 64%|######4   | 150M/233M [00: